<a href="https://colab.research.google.com/github/tanishq-0/Neuromatch/blob/main/silenthypo_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction & Hypotheses

Under 0% contrast—when no visual cue is present—mice must rely entirely on their internal expectations to decide which side to choose. Recent work (Teodorescu et al. 2023, preprint: https://www.biorxiv.org/content/10.1101/2023.07.04.547684v1) shows that 20–30% of brain regions encode these priors in their pre‑stimulus activity, with secondary motor cortex (MOs) among the strongest.

In this notebook, we examine whether MOs firing in the 0–0.5 s before stimulus onset predicts the trial’s block bias (probabilityLeft) and whether it carries information about the mouse’s eventual choice. Although choice decoding can often be achieved in downstream motor areas, our focus here is on the prior—what the mouse expects.

All sessions tagged with MOs were selected and, in this initial analysis, included all neurons on that probe rather than anatomically restricting to neurons from MOs. Future work should refine this selection and also extend it to other frontal regions (e.g., PL, ACCd).


## Hypotheses & Analyses

We ran four tests:

1. **Predict block bias (all blocks)**  
   - Use MOs firing 0–0.5 s before stimulus to predict `probabilityLeft` (0.2, 0.5, 0.8) on every trial.  
   - **Result**: 57.7 % accuracy (±10.6 %), p < 0.001.  

2. **Predict block bias (strong bias only)**  
   - Restrict to 0‑contrast trials with `probabilityLeft` = 0.2 or 0.8.  
   - **Result**: 58.5 % accuracy (±4.2 %), p < 0.001.  

3. **Predict choice (all contrasts)**  
   - Predict left vs right choice from pre‑stimulus activity over all contrasts.  
   - **Result**: 51.5 % accuracy (±3.4 %), p = 0.22 (not significant).  

4. **Predict choice (zero contrast)**  
   - Predict choice with no sensory evidence (contrast = 0).  
   - **Result**: 59.4 % accuracy (±7.4 %), p = 0.18 (not significant).  


## Interpretation

- MOs firing *before* the stimulus reliably reflects the block bias the mouse has learned—even with no visual cue.  
- Pre‑stimulus MOs activity does **not** reliably predict the actual choice, except weakly under zero contrast.  
- This supports the idea that **priors are encoded in frontal cortex**, while choice signals may rely on other regions or later processing stages.  


## Limitations

-  **Selected a single session** with MOs units and used **all sorted clusters** in that probe’s range, rather than sub-selecting only the most anterior or strictly “MOs‐only” channels.

- Results should be replicated across additional sessions and, ideally, across multiple frontal subregions (PL, ACCd) to confirm generality.

- **Pre-processing and dimensionality reduction** (e.g., PCA, regularization) were minimal—tuning these steps may enhance decoding performance.

- **Model hyperparameters** were left at defaults; systematic optimization (e.g., grid search, nested CV) could further boost accuracy.

- Increasing number of trials could also improve the models generalization.

In [ ]:
! pip install ONE-api
! pip install ibllib

  Using cached one_api-3.3.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached iblutil-1.20.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached boto3-1.39.12-py3-none-any.whl.metadata (6.7 kB)
  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached botocore-1.39.12-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.13.1-py3-none-any.whl.metadata (1.7 kB)
Using cached one_api-3.3.0-py3-none-any.whl (996 kB)
Using cached iblutil-1.20.0-py3-none-any.whl (43 kB)
Using cached boto3-1.39.12-py3-none-any.whl (139 kB)
Using cached botocore-1.39.12-py3-none-any.whl (13.9 MB)
Using cached colorlog-6.9.0-py3-none-any.whl (11 kB)
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 2.2 MB/s eta 0:00:00
  Using cached ibllib-3.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached flake8-7.3.0-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cache

In [ ]:
from one.api import ONE
ONE.setup(base_url='https://openalyx.internationalbrainlab.org', silent=True)
one = ONE(password='international')

Connected to https://openalyx.internationalbrainlab.org as user "intbrainlab"


In [ ]:
from one.api import ONE
one = ONE()

In [ ]:
brain_acronym = ['MOs']
# query sessions endpoint
sessions = one.search(atlas_acronym=brain_acronym, query_type='remote')
print(f'No. of detected sessions: {len(sessions)}')

# query insertions endpoint
insertions = one.search_insertions(atlas_acronym=brain_acronym)
print(f'No. of detected insertions: {len(insertions)}')

No. of detected sessions: 62
No. of detected insertions: 62


In [ ]:
eid = sessions[0]
from pprint import pprint
datasets = one.list_datasets(eid)
pprint(datasets)

['alf/#2024-05-06#/_ibl_wheel.position.npy',
 'alf/#2024-05-06#/_ibl_wheelMoves.peakAmplitude.npy',
 'alf/_ibl_bodyCamera.dlc.pqt',
 'alf/_ibl_bodyCamera.times.npy',
 'alf/_ibl_leftCamera.dlc.pqt',
 'alf/_ibl_leftCamera.features.pqt',
 'alf/_ibl_leftCamera.times.npy',
 'alf/_ibl_passiveGabor.table.csv',
 'alf/_ibl_passivePeriods.intervalsTable.csv',
 'alf/_ibl_passiveRFM.times.npy',
 'alf/_ibl_passiveStims.table.csv',
 'alf/_ibl_rightCamera.dlc.pqt',
 'alf/_ibl_rightCamera.features.pqt',
 'alf/_ibl_rightCamera.times.npy',
 'alf/_ibl_trials.goCueTrigger_times.npy',
 'alf/_ibl_trials.stimOff_times.npy',
 'alf/_ibl_trials.table.pqt',
 'alf/_ibl_wheel.timestamps.npy',
 'alf/_ibl_wheelMoves.intervals.npy',
 'alf/bodyCamera.ROIMotionEnergy.npy',
 'alf/bodyROIMotionEnergy.position.npy',
 'alf/leftCamera.ROIMotionEnergy.npy',
 'alf/leftROIMotionEnergy.position.npy',
 'alf/licks.times.npy',
 'alf/probe00/electrodeSites.brainLocationIds_ccf_2017.npy',
 'alf/probe00/electrodeSites.localCoordinate

In [ ]:
# Your session EIDs from insertions
session_eids = sessions

# Substrings you need somewhere in the dataset path
required = [
  '_ibl_trials.table.pqt',
  'spikes.times.npy',
  'spikes.clusters.npy',
  'clusters.uuids.csv'  # if present in this session
]

usable_eids = []
for eid in session_eids:
    try:
        available = one.list_datasets(eid)  # e.g. ['alf/_ibl_trials.contrastLeft.npy', ...]
        # Check that for each required substring, at least one available path contains it
        if all(any(sub in ds for ds in available) for sub in required):
            usable_eids.append(eid)
    except Exception:
        continue

print(f"Found {len(usable_eids)} sessions with all required datasets.")

Found 62 sessions with all required datasets.


In [ ]:
eid = usable_eids[0]  # start with the first usable session
print("Decoding session:", eid)
import pandas as pd

trials = one.load_object(eid, 'trials')

print("Total trials:", len(trials["probabilityLeft"]))
print("Atrributes of trials:", trials.keys())

Decoding session: cef05f87-161b-4031-932c-6f47daf89698


(S3) /root/Downloads/ONE/openalyx.internationalbrainlab.org/hausserlab/Subjects/PL033/2022-10-06/001/alf/_ibl_trials.goCueTrigger_times.npy: 100%|██████████| 4.82k/4.82k [00:00<00:00, 51.1kB/s]
(S3) /root/Downloads/ONE/openalyx.internationalbrainlab.org/hausserlab/Subjects/PL033/2022-10-06/001/alf/_ibl_trials.table.pqt: 100%|██████████| 49.0k/49.0k [00:00<00:00, 451kB/s]
(S3) /root/Downloads/ONE/openalyx.internationalbrainlab.org/hausserlab/Subjects/PL033/2022-10-06/001/alf/_ibl_trials.stimOff_times.npy: 100%|██████████| 4.82k/4.82k [00:00<00:00, 61.9kB/s]


Total trials: 586
Atrributes of trials: dict_keys(['goCueTrigger_times', 'stimOff_times', 'goCue_times', 'response_times', 'choice', 'stimOn_times', 'contrastLeft', 'contrastRight', 'probabilityLeft', 'feedback_times', 'feedbackType', 'rewardVolume', 'firstMovement_times', 'intervals'])


In [ ]:
for k, v in trials.items():
  print(k, v.shape)

goCueTrigger_times (586,)
stimOff_times (586,)
goCue_times (586,)
response_times (586,)
choice (586,)
stimOn_times (586,)
contrastLeft (586,)
contrastRight (586,)
probabilityLeft (586,)
feedback_times (586,)
feedbackType (586,)
rewardVolume (586,)
firstMovement_times (586,)
intervals (586, 2)


In [ ]:
intervals = trials['intervals']
print(intervals)

[[  64.97456856   67.50346127]
 [  68.08865712   70.83336054]
 [  71.40445884   73.69994492]
 ...
 [2754.46772568 2756.82412226]
 [2757.87921783 2760.26062109]
 [2760.86311941 2785.17353429]]


In [ ]:
trials['interval 1'] = intervals[:,0]
trials['interval 2'] = intervals[:,1]
del trials['intervals']
data = pd.DataFrame(trials)
data.head()

,goCueTrigger_times,stimOff_times,goCue_times,response_times,choice,stimOn_times,contrastLeft,contrastRight,probabilityLeft,feedback_times,feedbackType,rewardVolume,firstMovement_times,interval 1,interval 2
0,65.669766,67.003366,65.670599,65.955465,-1.0,65.750024,NaN,1.000,0.5,65.955570,1.0,1.5,65.813431,64.974569,67.503461
1,68.883458,70.333299,68.884211,69.283658,1.0,68.883344,0.25,NaN,0.5,69.283771,1.0,1.5,69.084431,68.088657,70.833361
2,71.886755,73.199946,71.887511,72.159554,-1.0,71.886577,NaN,0.250,0.5,72.159650,1.0,1.5,71.980431,71.404459,73.699945
3,74.736940,76.469719,74.738125,75.430042,-1.0,74.736792,NaN,0.125,0.5,75.430156,1.0,1.5,74.982431,74.269539,76.969748
4,79.066452,80.286637,79.067359,79.234052,-1.0,79.066259,NaN,0.250,0.5,79.234142,1.0,1.5,78.882431,77.521547,80.786659


In [ ]:
data['trial_id'] = data.index
# Reorder to make trial_id first
cols = ['trial_id'] + [col for col in data.columns if col != 'trial_id']
data = data[cols]
data.head()

,trial_id,goCueTrigger_times,stimOff_times,goCue_times,response_times,choice,stimOn_times,contrastLeft,contrastRight,probabilityLeft,feedback_times,feedbackType,rewardVolume,firstMovement_times,interval 1,interval 2
0,0,65.669766,67.003366,65.670599,65.955465,-1.0,65.750024,NaN,1.000,0.5,65.955570,1.0,1.5,65.813431,64.974569,67.503461
1,1,68.883458,70.333299,68.884211,69.283658,1.0,68.883344,0.25,NaN,0.5,69.283771,1.0,1.5,69.084431,68.088657,70.833361
2,2,71.886755,73.199946,71.887511,72.159554,-1.0,71.886577,NaN,0.250,0.5,72.159650,1.0,1.5,71.980431,71.404459,73.699945
3,3,74.736940,76.469719,74.738125,75.430042,-1.0,74.736792,NaN,0.125,0.5,75.430156,1.0,1.5,74.982431,74.269539,76.969748
4,4,79.066452,80.286637,79.067359,79.234052,-1.0,79.066259,NaN,0.250,0.5,79.234142,1.0,1.5,78.882431,77.521547,80.786659


# Prediciting block bias (probabilityLeft) from pre-stmulus activtiy with traisl of all contrast values

In [ ]:
spikes = one.load_object(eid, 'spikes')
spike_times    = spikes['times']
spike_clusters = spikes['clusters']

(S3) /root/Downloads/ONE/openalyx.internationalbrainlab.org/hausserlab/Subjects/PL033/2022-10-06/001/alf/probe00/pykilosort/#2024-05-06#/spikes.amps.npy: 100%|██████████| 181M/181M [00:01<00:00, 120MB/s]
(S3) /root/Downloads/ONE/openalyx.internationalbrainlab.org/hausserlab/Subjects/PL033/2022-10-06/001/alf/probe00/pykilosort/#2024-05-06#/spikes.clusters.npy: 100%|██████████| 90.7M/90.7M [00:00<00:00, 177MB/s]
(S3) /root/Downloads/ONE/openalyx.internationalbrainlab.org/hausserlab/Subjects/PL033/2022-10-06/001/alf/probe00/pykilosort/#2024-05-06#/spikes.depths.npy: 100%|██████████| 181M/181M [00:03<00:00, 50.7MB/s]
(S3) /root/Downloads/ONE/openalyx.internationalbrainlab.org/hausserlab/Subjects/PL033/2022-10-06/001/alf/probe00/pykilosort/#2024-05-06#/spikes.samples.npy: 100%|██████████| 181M/181M [00:00<00:00, 185MB/s]
(S3) /root/Downloads/ONE/openalyx.internationalbrainlab.org/hausserlab/Subjects/PL033/2022-10-06/001/alf/probe00/pykilosort/#2024-05-06#/spikes.templates.npy: 100%|████████

In [ ]:
import numpy as np
unit_ids = np.unique(spike_clusters)
print(f"Number of units: {len(unit_ids)}")

Number of units: 730


In [ ]:
stim_times0 = data['stimOn_times'].values

In [ ]:
def extract_pre_stim(X_times, X_clusters, units, stim_times, pre=0.5):
    n_trials = len(stim_times)
    n_units  = len(units)
    X = np.zeros((n_trials, n_units), dtype=float)
    spikes_by_unit = {u: X_times[X_clusters == u] for u in units}
    for i, t0 in enumerate(stim_times):
        start, end = t0 - pre, t0
        for j, u in enumerate(units):
            st = spikes_by_unit[u]
            X[i, j] = np.sum((st > start) & (st < end)) / pre  # firing rate
    return X

X = extract_pre_stim(spike_times, spike_clusters, unit_ids, stim_times0)
print("X shape:", X.shape)  # should be (n_trials, n_units)

X shape: (586, 730)


In [ ]:
data_filtered = data[data['probabilityLeft'].isin([0.2, 0.8, 0.5])].reset_index(drop=True)
stim_times0 = data_filtered['stimOn_times'].values
y = data_filtered['probabilityLeft'].map({0.8: 0, 0.5: 1, 0.2:2}).values

In [ ]:
print("Shape of X:", X.shape)  # (n_trials, n_units)
print("Label distribution:", np.bincount(y))
print(y.shape)

Shape of X: (586, 730)
Label distribution: [264  90 232]
(586,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

clf = LogisticRegression(max_iter=1000)
scores = cross_val_score(clf, X, y, cv=5)

print(f"Mean accuracy: {scores.mean():.3f} ± {scores.std():.3f}")

Mean accuracy: 0.577 ± 0.106


In [ ]:
clf.fit(X, y)
weights = clf.coef_[0]  # one weight per neuron
# Get top 10 positively and negatively weighted units
pos_idx = np.argsort(weights)[-10:]
neg_idx = np.argsort(weights)[:10]

print("Top + units:", pos_idx)
print("Top – units:", neg_idx)

Top + units: [382 601 619 241 259 215 441 638 595 565]
Top – units: [504 265 667 703 574 655 567 493  68 526]


In [ ]:
from sklearn.model_selection import cross_val_score

n_perm = 50

# Actual model performance
actual_score = cross_val_score(clf, X, y, cv=5, n_jobs=-1).mean()

# Permutation scores (parallelized)
def permuted_score(_):
    y_perm = np.random.permutation(y)
    return cross_val_score(clf, X, y_perm, cv=5, n_jobs=-1).mean()

# Use joblib for parallelism
from joblib import Parallel, delayed
perm_scores = Parallel(n_jobs=-1)(delayed(permuted_score)(i) for i in range(n_perm))

# Compute p-value
p_val = np.mean(np.array(perm_scores) >= actual_score)
print(f"Permutation p‑value: {p_val:.3f}")

Permutation p‑value: 0.000


# Prediciting block bias (probabilityLeft) either 0.8 or 0.2 from pre-stimulus activity for trials with all contrast values

In [ ]:
data_filtered1 = data[data['probabilityLeft'].isin([0.2, 0.8])].reset_index(drop=True)
stim_times1 = data_filtered1['stimOn_times'].values
y = data_filtered1['probabilityLeft'].map({0.8: 0, 0.2: 1}).values

In [ ]:
def extract_pre_stim(X_times, X_clusters, units, stim_times, pre=0.5):
    n_trials = len(stim_times)
    n_units  = len(units)
    X = np.zeros((n_trials, n_units), dtype=float)
    spikes_by_unit = {u: X_times[X_clusters == u] for u in units}
    for i, t0 in enumerate(stim_times):
        start, end = t0 - pre, t0
        for j, u in enumerate(units):
            st = spikes_by_unit[u]
            X[i, j] = np.sum((st > start) & (st < end)) / pre  # firing rate
    return X

X = extract_pre_stim(spike_times, spike_clusters, unit_ids, stim_times1)
print("X shape:", X.shape)  # should be (n_trials, n_units)

X shape: (496, 730)


In [ ]:
print("Shape of X:", X.shape)  # (n_trials, n_units)
print("Label distribution:", np.bincount(y))
print("Shape of y:",y.shape)

Shape of X: (496, 730)
Label distribution: [264 232]
Shape of y: (496,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

clf = LogisticRegression(max_iter=1000)
scores = cross_val_score(clf, X, y, cv=5)

print(f"Mean accuracy: {scores.mean():.3f} ± {scores.std():.3f}")

Mean accuracy: 0.585 ± 0.042


In [ ]:
clf.fit(X, y)
weights = clf.coef_[0]  # one weight per neuron
# Get top 10 positively and negatively weighted units
pos_idx = np.argsort(weights)[-10:]
neg_idx = np.argsort(weights)[:10]

print("Top + units:", pos_idx)
print("Top – units:", neg_idx)

Top + units: [567  86 655 435 667 703  62 265 574 504]
Top – units: [565 313 595 638 215 441 259 601 417  92]


In [ ]:
from sklearn.model_selection import cross_val_score

n_perm = 50

# Actual model performance
actual_score = cross_val_score(clf, X, y, cv=5, n_jobs=-1).mean()

# Permutation scores (parallelized)
def permuted_score(_):
    y_perm = np.random.permutation(y)
    return cross_val_score(clf, X, y_perm, cv=5, n_jobs=-1).mean()

# Use joblib for parallelism
from joblib import Parallel, delayed
perm_scores = Parallel(n_jobs=-1)(delayed(permuted_score)(i) for i in range(n_perm))

# Compute p-value
p_val = np.mean(np.array(perm_scores) >= actual_score)
print(f"Permutation p‑value: {p_val:.3f}")

Permutation p‑value: 0.000


# Prediciting choice from pre-stimulus activity with all contrast values


In [ ]:
def extract_pre_stim(X_times, X_clusters, units, stim_times, pre=0.5):
    n_trials = len(stim_times)
    n_units  = len(units)
    X = np.zeros((n_trials, n_units), dtype=float)
    spikes_by_unit = {u: X_times[X_clusters == u] for u in units}
    for i, t0 in enumerate(stim_times):
        start, end = t0 - pre, t0
        for j, u in enumerate(units):
            st = spikes_by_unit[u]
            X[i, j] = np.sum((st > start) & (st < end)) / pre  # firing rate
    return X

X = extract_pre_stim(spike_times, spike_clusters, unit_ids, stim_times0)
print("X shape:", X.shape)  # should be (n_trials, n_units)

X shape: (586, 730)


In [ ]:
data_filtered = data[data['choice'].isin([-1.0,1.0])].reset_index(drop=True)
stim_times0 = data_filtered['stimOn_times'].values
y = data_filtered['choice'].map({-1.0: 0, 1.0: 1}).values
print("Shape of X:", X.shape)  # (n_trials, n_units)
print("Label distribution:", np.bincount(y))
print(y.shape)

Shape of X: (586, 730)
Label distribution: [333 253]
(586,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

clf = LogisticRegression(max_iter=1000)
scores = cross_val_score(clf, X, y, cv=5)

print(f"Mean accuracy: {scores.mean():.3f} ± {scores.std():.3f}")

Mean accuracy: 0.515 ± 0.034


In [ ]:
clf.fit(X, y)
weights = clf.coef_[0]  # one weight per neuron
# Get top 10 positively and negatively weighted units
pos_idx = np.argsort(weights)[-10:]
neg_idx = np.argsort(weights)[:10]

print("Top + units:", pos_idx)
print("Top – units:", neg_idx)

Top + units: [725 518 656 376 115 460  32 417  92 300]
Top – units: [667 375 433 402  67 201 483 620  89 681]


In [ ]:
from sklearn.model_selection import cross_val_score

n_perm = 50

# Actual model performance
actual_score = cross_val_score(clf, X, y, cv=5, n_jobs=-1).mean()

# Permutation scores (parallelized)
def permuted_score(_):
    y_perm = np.random.permutation(y)
    return cross_val_score(clf, X, y_perm, cv=5, n_jobs=-1).mean()

# Use joblib for parallelism
from joblib import Parallel, delayed
perm_scores = Parallel(n_jobs=-1)(delayed(permuted_score)(i) for i in range(n_perm))

# Compute p-value
p_val = np.mean(np.array(perm_scores) >= actual_score)
print(f"Permutation p‑value: {p_val:.3f}")

Permutation p‑value: 0.220


# Prediciting choice from pre-stimuls activity with only 0 contrast trials

In [ ]:
# Select only trials where both contrasts are zero
mask0 = (data['contrastLeft'] == 0.0) | (data['contrastRight'] == 0.0)
tr0 = data[mask0].reset_index(drop=True)

print(f"Using {len(tr0)} zero‑contrast trials out of {len(data)} total.")
tr0.head()

Using 74 zero‑contrast trials out of 586 total.


,trial_id,goCueTrigger_times,stimOff_times,goCue_times,response_times,choice,stimOn_times,contrastLeft,contrastRight,probabilityLeft,feedback_times,feedbackType,rewardVolume,firstMovement_times,interval 1,interval 2
0,19,140.251848,143.115020,140.252673,141.071847,1.0,140.251707,NaN,0.0,0.5,141.072658,-1.0,0.0,140.613431,137.808051,143.615045
1,23,155.748159,156.964635,155.749090,155.899059,1.0,155.747990,0.0,NaN,0.5,155.899142,1.0,1.5,155.513431,155.068558,157.464763
2,25,162.984454,165.484250,162.985300,164.420146,-1.0,162.984267,NaN,0.0,0.5,164.420256,1.0,1.5,164.289431,161.347064,165.984340
3,37,213.633558,214.999831,213.634434,213.942458,-1.0,213.633335,NaN,0.0,0.5,213.942537,1.0,1.5,213.818431,213.067558,215.499860
4,43,237.532947,238.679463,237.533678,237.624347,-1.0,237.532778,NaN,0.0,0.5,237.624435,1.0,1.5,237.454431,235.235141,239.179553


In [ ]:
stim_times2 = tr0['stimOn_times'].values
print(len(stim_times2))

74


In [ ]:
def extract_pre_stim(X_times, X_clusters, units, stim_times, pre=0.5):
    n_trials = len(stim_times)
    n_units  = len(units)
    X = np.zeros((n_trials, n_units), dtype=float)
    spikes_by_unit = {u: X_times[X_clusters == u] for u in units}
    for i, t0 in enumerate(stim_times):
        start, end = t0 - pre, t0
        for j, u in enumerate(units):
            st = spikes_by_unit[u]
            X[i, j] = np.sum((st > start) & (st < end)) / pre  # firing rate
    return X

X = extract_pre_stim(spike_times, spike_clusters, unit_ids, stim_times2)
print("X shape:", X.shape)  # should be (n_trials, n_units)

X shape: (74, 730)


In [ ]:
y = tr0['choice'].map({-1.0: 0, 1.0: 1}).values
print("Shape of X:", X.shape)  # (n_trials, n_units)
print("Label distribution:", np.bincount(y))
print(y.shape)

Shape of X: (74, 730)
Label distribution: [44 30]
(74,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

clf = LogisticRegression(max_iter=1000)
scores = cross_val_score(clf, X, y, cv=5)

print(f"Mean accuracy: {scores.mean():.3f} ± {scores.std():.3f}")

Mean accuracy: 0.594 ± 0.074


In [ ]:
clf.fit(X, y)
weights = clf.coef_[0]  # one weight per neuron
# Get top 10 positively and negatively weighted units
pos_idx = np.argsort(weights)[-10:]
neg_idx = np.argsort(weights)[:10]

print("Top + units:", pos_idx)
print("Top – units:", neg_idx)

Top + units: [619 365 502 601 266 653 227 252  20  14]
Top – units: [492 486 395  51 230 427 478  62 368 189]


In [ ]:
from sklearn.model_selection import cross_val_score

n_perm = 50

# Actual model performance
actual_score = cross_val_score(clf, X, y, cv=5, n_jobs=-1).mean()

# Permutation scores (parallelized)
def permuted_score(_):
    y_perm = np.random.permutation(y)
    return cross_val_score(clf, X, y_perm, cv=5, n_jobs=-1).mean()

# Use joblib for parallelism
from joblib import Parallel, delayed
perm_scores = Parallel(n_jobs=-1)(delayed(permuted_score)(i) for i in range(n_perm))

# Compute p-value
p_val = np.mean(np.array(perm_scores) >= actual_score)
print(f"Permutation p‑value: {p_val:.3f}")

Permutation p‑value: 0.180
